In [2]:
import pickle
import pandas as pd 
import numpy as np
import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  

In [3]:
with open('./data_output/final/prob/fb/small_df', 'rb') as handle:
    small_df = pickle.loads(handle.read())

In [4]:
w_feature = []
for i in range(0,100):
    w_feature.append('FN1_wv_'+str(i))
    w_feature.append('FN2_wv_'+str(i))
    
    
character_feature=[]
for index, x in enumerate(small_df.columns.values):
    if 'FirstName' in x and '_' in x:
        character_feature.append(x)
print('len of character_feature:',len(character_feature))

radial_feature = []
for index, x in enumerate(small_df.columns.values):
    if 'FN1_radical_' in x or 'FN2_radical_' in x:
        radial_feature.append(x)

print('len of radial_feature:',len(radial_feature))

in_feature=[]
for index, x in enumerate(small_df.columns.values):
    #if 'FN1_muin_' in x or 'FN2_muin_' in x or 'FN1_sonin_' in x or 'FN2_sonin_' in x:
    if 'FN1_mu_in_' in x or 'FN2_mu_in_' in x or 'FN1_sonin_' in x or 'FN2_sonin_' in x or 'FN2_in_' in x or 'FN1_in_' in x:
        in_feature.append(x)
print('len of in_feature:',len(in_feature))

LN_feature = []
for f in small_df.columns.values:
    if 'LastName_' in f:
        LN_feature.append(f)
print('len of LN_feature:',len(LN_feature))

len of character_feature: 2632
len of radial_feature: 340
len of in_feature: 139
len of LN_feature: 391


# 取得筆劃與五格 

In [5]:
small_df['FirstName1'] = small_df.FirstName.apply(lambda x:x[0])
small_df['FirstName2'] = small_df.FirstName.apply(lambda x:x[1])

In [6]:
from mylib.Taiwan_name_seperate import get_LastName_from_FN
small_df['LastName'] = get_LastName_from_FN(small_df['FirstName'].tolist() ,small_df['name'].tolist() )

In [7]:
from mylib.fortune_calculator import Get_stroke
from mylib.fortune_calculator import get_stroke_state
from mylib.fortune_calculator import get_talent_state
from mylib.fortune_calculator import get_talent_type
from mylib.fortune_calculator import test_name_Fortune_telling
from mylib.fortune_calculator import stroke_earth
from mylib.fortune_calculator import stroke_heaven
from mylib.fortune_calculator import stroke_man
from mylib.fortune_calculator import stroke_outside
from mylib.fortune_calculator import stroke_total

In [8]:
test_name_Fortune_telling("林書豪")

名字： 林書豪
五格
天格： 9 凶
地格： 24 吉
人格： 18 吉
外格： 15 吉
總格： 32 吉
三才
天才: 水
人才: 金
地才: 火
三才格局： 凶多於吉



# 加入三才五格feature

In [9]:
%%time

FN_list = small_df['FirstName'].tolist()
LN_list = small_df['LastName'].tolist()

heaven_list = []
earth_list = []
man_list = []
outside_list = []
total_list = []
talent_state_list = []

row =  small_df [ small_df.name.apply(lambda x:x=='王建民') ].iloc[0]

for index,FN in enumerate(FN_list):

    row['FirstName'] = FN
    row['LastName'] = LN_list[index]
    #print(FN,row['LastName'])
    heaven = stroke_heaven (row['LastName'])
    earth = stroke_earth (row['FirstName'])
    man = stroke_man (row)
    outside = stroke_outside (row)
    total = stroke_total(row)
    
    heaven_list.append(get_stroke_state(heaven))
    earth_list.append(get_stroke_state(earth))
    man_list.append(get_stroke_state(man))
    outside_list.append(get_stroke_state(outside))
    total_list.append(get_stroke_state(total))
    
    
    three_talent = get_talent_type(heaven) + get_talent_type(man) + get_talent_type(earth)
    if '不明' in three_talent:
        print(FN,'=',row['LastName'],len(FN),len(row['LastName']))
    talent_state_list.append( get_talent_state(three_talent) )
    
    
small_df['天格'] = heaven_list
small_df['地格'] = earth_list
small_df['人格'] = man_list
small_df['外格'] = outside_list
small_df['總格'] = total_list
small_df['三才'] = talent_state_list

%xdel heaven_list 
%xdel earth_list
%xdel man_list
%xdel outside_list
%xdel total_list
%xdel talent_state_list

Wall time: 21.6 s


In [17]:
small_df[ ['name','天格','地格','人格','外格','總格','三才']]

,name,天格,地格,人格,外格,總格,三才
0,張家琪,凶,吉,吉,凶,凶,大吉
1,涂銘峰,凶,吉,吉,吉,吉,中吉
2,歐昱成,吉,吉,吉,吉,吉帶凶,大吉
5,張耕耀,凶,吉帶凶,吉,吉,吉,吉多於凶
6,胡小芳,凶,吉,凶,吉,吉,中吉
7,張朝棟,凶,吉,吉,吉,吉,中吉
8,卓裕偉,凶,吉,吉,凶,吉,大吉
9,唐妍,吉,吉,凶,吉,吉,凶多於吉
10,林健銘,凶,吉,凶,吉,吉,大凶
12,陳冠彰,吉,吉,吉,吉,吉,大吉


In [13]:
import sys
#check_memory_usage():
# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
object_size = sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)
for index,obj in enumerate(object_size):
    if index>10:
        break
    print('FileName:',obj[0],'FileSize:',obj[1]/1000000,'MB')

FileName: small_df FileSize: 721.107049 MB
FileName: FN_list FileSize: 1.222368 MB
FileName: LN_list FileSize: 1.222368 MB
FileName: row FileSize: 0.667101 MB
FileName: character_feature FileSize: 0.023728 MB
FileName: LN_feature FileSize: 0.003304 MB
FileName: radial_feature FileSize: 0.002896 MB
FileName: w_feature FileSize: 0.001672 MB
FileName: in_feature FileSize: 0.001248 MB
FileName: MLPClassifier FileSize: 0.001056 MB
FileName: StandardScaler FileSize: 0.001056 MB


In [12]:
small_df = pd.get_dummies(small_df, columns=["天格"])
small_df = pd.get_dummies(small_df, columns=["地格"])
small_df = pd.get_dummies(small_df, columns=["人格"])
small_df = pd.get_dummies(small_df, columns=["外格"])
small_df = pd.get_dummies(small_df, columns=["總格"])
small_df = pd.get_dummies(small_df, columns=["三才"])

# 再加上名字比例

In [14]:
with open('./data_output/intersection_name_dict.txt', 'rb') as handle:
    intersection_name_dict = pickle.loads(handle.read())

with open('./data_output/LF_common_char_intersection_ratio_dict.txt', 'rb') as handle:
    LF_common_char_intersection_ratio_dict = pickle.loads(handle.read())


In [15]:
from mylib.PrintException  import PrintException
from mylib.is_chinese import is_chinese
from mylib.is_number import is_number

from mylib.Taiwan_name_seperate import GetFirstName
from mylib.Taiwan_name_seperate import GetLastName
from mylib.Taiwan_name_seperate import checkLastName

#find 名字是否含非繁體字會用之字的簡體字
from mylib.findSimName import findSimName

In [16]:
def DropSimplifiedName(cname_df):
    return cname_df[cname_df.name.apply(lambda x: findSimName(x)==False )]

In [22]:
def name_char_add_to_dic(name,i):
    for c in name:
        if(c in name_rate_dic_list[i] ):
            name_rate_dic_list[i][c]+= 1
        else:
            name_rate_dic_list[i][c] = 1 

            
def name_evolve_5(name_rate_dic_list, data_df,Exam):
    #1940~2005  5歲一區間
    
    for i in range(0,14):
        ocr_total_char = 0
        for j in range(5):
            orc_year = 1958+i*5+j
            FB_year = 1940+i*5+j
            no_orc_year = False
            if str(orc_year) in Missing_year_list or orc_year>2000:
                no_orc_year = True

            #Total_df時Exam=True,1976~1982不看OCR
            if Exam and orc_year>=1994 and orc_year<=2000:
                no_orc_year = True
                
            if(no_orc_year==False):
                for  c in year_character_dic[str(orc_year)]:
                    ocr_total_char+=year_character_dic[str(orc_year)][c]
                    if(c in name_rate_dic_list[i] ):
                        name_rate_dic_list[i][c]+= year_character_dic[str(orc_year)][c]
                    else:
                        name_rate_dic_list[i][c] = year_character_dic[str(orc_year)][c]

        FB_year = 1940+i*5
        FB_name_list = data_df[data_df.message.apply(lambda x: x>=FB_year and x<FB_year+5)].FirstName.tolist()
        FB_total_char=0
        
        tmp_dic = {}

        for name in FB_name_list:
            FB_total_char+=len(name)
            for c in name:
                if(c in name_rate_dic_list[i] ):
                    name_rate_dic_list[i][c]+= 1
                else:
                    name_rate_dic_list[i][c] = 1        
                    
        #根據此字出現在姓和名中的比例來調整次數    
        Total_char = 0
        for c in name_rate_dic_list[i]:
            char = c[0]
            if(char in LF_common_char_intersection_ratio_dict): 
                 name_rate_dic_list[i][char] = name_rate_dic_list[i][char] * (1- LF_common_char_intersection_ratio_dict[char]) 
            Total_char+=name_rate_dic_list[i][char]
            
        print(1940+i*5,"DF:",FB_total_char,"OCR:",ocr_total_char,"Total:",Total_char)
        
        for c in name_rate_dic_list[i]:
            name_rate_dic_list[i][c]/= Total_char
    

In [23]:
# -*- coding: utf-8 -*-
import os

#得取OCR 出來的榜單資料
#1958-2000年榜單　37份（缺6年）　1988　1989 1990　1991　1992　1994　
#1940-1982　出生
#77歲-35歲

Path ='../univeristyName/'
year_character_dic = {}
Missing_year_list = ['1988','1989','1990','1991','1992','1994']
for root, dirs, fileNames in os.walk(Path):
    if fileNames ==[]:
        continue
    for fileName in fileNames:
        try:
            with open(root+"/"+fileName ,'r',encoding = 'utf-8') as F:
                character_dic = {}
                #一行一行讀
                for line in F.readlines():
                    #print(line)
                    for character in line:
                        if is_chinese(character)==True and findSimName(character)==False:
                            if (character in character_dic):
                                character_dic[character]+=1
                            else:
                                character_dic[character] = 1 
                year_character_dic[fileName[:4]] = character_dic
            
        except Exception as e:
            print (e)
            print (fileName)
            PrintException() 

In [24]:
remain_year_list=[7,8,9,10,11,12,13]

In [26]:
with open('./data_output/final/prob/fb/train_index', 'rb') as handle:
    train_index = pickle.loads(handle.read())
with open('./data_output/final/prob/fb/test_index', 'rb') as handle:
    test_index = pickle.loads(handle.read())
with open('./data_output/final/prob/fb/random_avg_year_error', 'rb') as handle:
    random_avg_year_error = pickle.loads(handle.read())  

In [27]:
small_df['message'] = small_df.message.apply(lambda x:int(x))
Total_real_name_rate_dic_list = [ {} for x in range(14)]
name_evolve_5(Total_real_name_rate_dic_list,small_df.loc[train_index],False)

1940 DF: 0 OCR: 68794 Total: 48662.4898828
1945 DF: 0 OCR: 157478 Total: 110916.994853
1950 DF: 0 OCR: 244841 Total: 175104.023014
1955 DF: 0 OCR: 225425 Total: 162883.874234
1960 DF: 0 OCR: 266349 Total: 199943.384056
1965 DF: 0 OCR: 349740 Total: 249045.073138
1970 DF: 0 OCR: 0 Total: 0
1975 DF: 30542 OCR: 348568 Total: 296128.520249
1980 DF: 30300 OCR: 613915 Total: 488399.481267
1985 DF: 30160 OCR: 0 Total: 29131.1415557
1990 DF: 30446 OCR: 0 Total: 29557.3070146
1995 DF: 30906 OCR: 0 Total: 30175.8323063
2000 DF: 30724 OCR: 0 Total: 30053.1762173
2005 DF: 30792 OCR: 0 Total: 30193.8258536


In [28]:
Total_real_name_rate_dic_list[10]['義']

0.0012470295525631064

In [29]:
FN1_list = small_df['FirstName1'].tolist()
FN2_list = small_df['FirstName2'].tolist()

for i in remain_year_list:
    ratio_list = []
    for c in FN1_list:
        ratio = 0
        if c in Total_real_name_rate_dic_list[i]:
            ratio = Total_real_name_rate_dic_list[i][c]
        ratio_list.append( ratio )
    small_df['FirstName1_ratio_'+str(i)] = ratio_list
    
    ratio_list = []
    for c in FN2_list:
        ratio = 0
        if c in Total_real_name_rate_dic_list[i]:
            ratio = Total_real_name_rate_dic_list[i][c]
        ratio_list.append( ratio )
    small_df['FirstName2_ratio_'+str(i)] = ratio_list  
    
    
%xdel FN1_list
%xdel FN2_list

In [30]:
small_df.head(5)

,name,message,userID,FirstName,FN1,FN2,FN1_muin,FN2_muin,BirthYear,GuessedGender,...,FirstName1_ratio_9,FirstName2_ratio_9,FirstName1_ratio_10,FirstName2_ratio_10,FirstName1_ratio_11,FirstName2_ratio_11,FirstName1_ratio_12,FirstName2_ratio_12,FirstName1_ratio_13,FirstName2_ratio_13
0,張家琪,1984,1216132381818443,家琪,29,384,iā,í,8,0,...,0.008207,0.003150,0.008254,0.003307,0.010131,0.002942,0.007955,0.003086,0.007788,0.003733
1,涂銘峰,1983,1381624175192111,銘峰,44,233,íng,ēng,8,1,...,0.005595,0.001991,0.004229,0.002470,0.004673,0.002055,0.003693,0.001231,0.002053,0.001424
2,歐昱成,1980,1442601205791441,昱成,97,113,ù,éng,8,1,...,0.002025,0.001745,0.003857,0.002110,0.005004,0.001812,0.005224,0.001915,0.005167,0.000890
5,張耕耀,1983,1592555917430952,耕耀,1679,12,ēng,ào,8,1,...,0.000103,0.001672,0.000202,0.001715,0.000231,0.001450,0.000133,0.001489,0.000099,0.000757
6,胡小芳,1984,1384729524883556,小芳,53,246,iǎo,āng,8,0,...,0.025080,0.003732,0.008089,0.003509,0.002462,0.003371,0.002250,0.002588,0.004732,0.003600


In [31]:
#輸出完整處理過後的FB name
with open('./data_output/final/prob/all_in/final_FB_name_df', 'wb') as handle:
     pickle.dump(small_df, handle)